In [8]:
import pickle
import torch
from torch_geometric.data import Data

In [9]:
path_labels = './data/dict/data_y_bin_v2.pkl'
with open(path_labels, 'rb') as fp:
    dic_label = pickle.load(fp)

with open('./data/dict/adm_info_v2.pkl', 'rb') as fp:
    demo_dict = pickle.load(fp)

# import dictionnaire pickle dic_global
with open('./data/dict/dic_global_v2.pkl', 'rb') as fp:
    dic_global = pickle.load(fp)

with open('./data/dict/data_raw_v2.pkl', 'rb') as fp:
    data_x = pickle.load(fp)

In [10]:
def node_edge_gen(seq, dic_global, liste_node, liste_type):
    indices_d = list(set(seq['diagnoses']))
    indices_p = list(set(seq['procedures']))
    indices_m = list(set(seq['drugs']))
    # indices_l = list(set(seq['labevents']))

    for diag in indices_d:
        node = int(dic_global[diag]['identifiant']) + 1
        type_node = int(dic_global[diag]['type_code'])
        liste_node.append(node)
        liste_type.append(type_node)
    
    for proc in indices_p:
        node = int(dic_global[proc]['identifiant']) + 1
        type_node = int(dic_global[proc]['type_code'])
        liste_node.append(node)
        liste_type.append(type_node)
    
    for med in indices_m:
        node = int(dic_global[med]['identifiant']) + 1
        type_node = int(dic_global[med]['type_code'])
        liste_node.append(node)
        liste_type.append(type_node)
    
    # for lab in indices_l:
    #     node = int(dic_global[lab]['identifiant']) + 1
    #     type_node = int(dic_global[lab]['type_code'])
    #     liste_node.append(node)
    #     liste_type.append(type_node)
        
    return liste_node, liste_type

def creation_edge_index(x, liste_type, dic_type):
    # Edges (graphe complet)
    edge_attr = []
    edge_index = []
    all_edges = []

    for i in range(len(x)):
        for j in range(i+1,len(x)):
            all_edges.append((i, j))
            edge_attr.append(dic_type[(liste_type[i], liste_type[j])])
    source, target = zip(*all_edges)

    edge_index = torch.tensor([source, target], dtype=torch.int64)
    edge_attr = torch.tensor(edge_attr, dtype=torch.int64)

    return edge_index , edge_attr

In [11]:
num_nodes = len(dic_global) + 1

In [12]:
num_nodes

14444

In [13]:
dic_type = {(0,0): 0,
            (0,1): 0,
            (1,0): 0, 
            (0,2): 0,
            (2,0): 0,
            (0,3): 0,
            (3,0): 0,
            (1,1): 1,
            (1,2): 2,
            (2,1): 2,
            (1,3): 3,
            (3,1): 3,
            (2,2): 4,
            (2,3): 5,
            (3,2): 5,
            (3,3): 6
            }

In [15]:
len(dic_type)

16

In [16]:
from tqdm import tqdm

In [17]:
liste_dataset = []

for subj_id, vitsits_demo in tqdm(demo_dict.items()):
    # print(subj_id, vitsits_demo)
    liste_patient = []
    label = torch.tensor(dic_label[subj_id], dtype=torch.int64)
    for hadm_id, info in vitsits_demo.items():
        visite_objet = Data()
        visite_objet.subject_id = torch.tensor([subj_id], dtype=torch.int64)
        visite_objet.hadm_id = torch.tensor([hadm_id], dtype=torch.int64)
        visite_objet.age = torch.tensor([info['age']], dtype=torch.int64)
        visite_objet.time = torch.tensor([info['admittime']], dtype=torch.int64)
        visite_objet.rang = torch.tensor([info['rang']], dtype=torch.int64)
        visite_objet.type = torch.tensor([info['adm_type']], dtype=torch.int64)
        
        liste_node ,liste_type = [], []
        liste_node, liste_type = node_edge_gen(data_x[subj_id][hadm_id], dic_global,liste_node,liste_type)
        visite_objet.x = torch.cat([torch.tensor([0]), torch.tensor(liste_node, dtype=torch.int64)]).reshape(1,-1)
        liste_node = [0] + liste_node
        liste_type = [0] + liste_type
        
        edge_index, edge_attr = creation_edge_index(liste_node, liste_type, dic_type) 
        visite_objet.edge_index = edge_index
        visite_objet.edge_attr = edge_attr
        liste_patient.append(visite_objet)
    liste_patient.append(label)
    liste_dataset.append(liste_patient)

100%|██████████| 41419/41419 [04:36<00:00, 149.99it/s]


In [18]:
with open('./data/dic_type_correspondance_v2.pkl', 'wb') as fp:
    pickle.dump(dic_type, fp)

with open('./data/data_v2.pkl', 'wb') as fp:
    pickle.dump(liste_dataset, fp)

In [52]:
liste_patient

[Data(subject_id=[1], hadm_id=[1], age=[1], time=[1], rang=[1], type=[1], x=[1, 80], edge_index=[2, 3160], edge_attr=[3160]),
 Data(subject_id=[1], hadm_id=[1], age=[1], time=[1], rang=[1], type=[1], x=[1, 71], edge_index=[2, 2485], edge_attr=[2485]),
 Data(subject_id=[1], hadm_id=[1], age=[1], time=[1], rang=[1], type=[1], x=[1, 62], edge_index=[2, 1891], edge_attr=[1891]),
 tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0])]